# Análisis de Desastres Naturales (EM-DAT) con POO + Herencia

En este notebook vamos a:

1. Cargar y limpiar el dataset (Excel).
2. Definir clases POO para organizar el análisis.
3. Implementar **herencia** usando una clase base `AnalisisBase` que concentra validaciones comunes:
   - Verifica columnas requeridas
   - Convierte columnas numéricas de forma segura
4. Crear subclases para cada análisis del menú:
   - Muertes por tipo
   - Pérdidas económicas por tipo
   - Impacto por subtipo (muertes vs afectados)
   - K-Means por tipo (clusterización)
5. Visualizar los resultados con una clase `Visualizador`.


## 1) Imports + carga del Excel + limpieza base

- Importamos librerías.
- Cargamos el Excel con `pandas`.
- Convertimos columnas numéricas importantes.
- Detectamos automáticamente la columna de daños/pérdidas (si existe).
- Quitamos filas que no tengan `Country` o `Disaster Type` (porque son claves para los análisis).


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# scikit-learn (KMeans y escalado)
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# --- Cargar datos ---
df = pd.read_excel("public_emdat_2025-12-09.xlsx")

# Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# Columnas numéricas típicas (ajusta si tu Excel usa otros nombres)
columnas_numericas = ["Total Deaths", "Total Affected"]

for col in columnas_numericas:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

# Detectar automáticamente la columna de pérdidas económicas
col_danios = None
for col in df.columns:
    if "Damage" in col or "Damages" in col:
        col_danios = col
        df[col_danios] = pd.to_numeric(df[col_danios], errors="coerce").fillna(0)
        break

print("✅ Columna de daños detectada:", col_danios)

# Eliminar registros sin país o tipo de desastre
df = df.dropna(subset=["Country", "Disaster Type"])

df.head()


✅ Columna de daños detectada: Insured Damage ('000 US$)


,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update
0,1999-9388-DJI,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,DJI,...,NaN,NaN,0.0,NaN,NaN,NaN,56.446576,"[{""adm1_code"":1093,""adm1_name"":""Ali Sabieh""},{...",2006-03-01,2023-09-25
1,1999-9388-SDN,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,SDN,...,NaN,NaN,0.0,NaN,NaN,NaN,54.895152,"[{""adm1_code"":2757,""adm1_name"":""Northern Darfu...",2006-03-08,2023-09-25
2,1999-9388-SOM,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,SOM,...,NaN,NaN,0.0,NaN,NaN,NaN,54.895152,"[{""adm1_code"":2691,""adm1_name"":""Bay""},{""adm1_c...",2006-03-08,2023-09-25
3,2000-0001-AGO,No,tec-tra-roa-roa,Technological,Transport,Road,Road,NaN,NaN,AGO,...,NaN,NaN,0.0,NaN,NaN,NaN,54.895152,NaN,2004-10-27,2023-09-25
4,2000-0002-AGO,No,nat-hyd-flo-riv,Natural,Hydrological,Flood,Riverine flood,NaN,NaN,AGO,...,NaN,NaN,0.0,NaN,10000.0,18217.0,54.895152,"[{""adm2_code"":4214,""adm2_name"":""Baia Farta""},{...",2005-02-03,2023-09-25
